# NLP Sentiment Analysis for Earnings Calls

This notebook demonstrates how to use transformer-based NLP models for analyzing earnings call transcripts and extracting trading signals.

## Key Concepts
- **FinBERT**: BERT model fine-tuned on financial text
- **Sentiment Classification**: Positive, Negative, Neutral for financial statements
- **Trading Signals**: Convert sentiment scores to actionable predictions

## Requirements
```bash
pip install torch transformers numpy pandas matplotlib
```

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Load FinBERT Model

In [ ]:
# Load pre-trained FinBERT model
MODEL_NAME = "ProsusAI/finbert"

print("Loading FinBERT model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model = model.to(device)
model.eval()

print(f"Model loaded: {MODEL_NAME}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

## 2. Sample Earnings Call Statements

We'll use synthetic examples representing typical earnings call language.

In [ ]:
# Sample earnings call statements (synthetic examples)
SAMPLE_STATEMENTS = [
    # Positive statements
    "We are pleased to report record revenue growth of 25% year-over-year, driven by strong demand across all segments.",
    "Our margins expanded significantly due to operational efficiencies and favorable product mix.",
    "Customer acquisition exceeded expectations, with net new customers up 40% from last quarter.",
    "We're raising our full-year guidance and expect continued momentum in the second half.",
    "Free cash flow generation was exceptional, allowing us to accelerate our share buyback program.",
    
    # Negative statements
    "We faced significant headwinds from supply chain disruptions that impacted our ability to meet demand.",
    "Operating expenses increased substantially due to higher raw material costs and wage inflation.",
    "Customer churn rates rose in the quarter, primarily in our enterprise segment.",
    "We are lowering our guidance for the remainder of the year due to macroeconomic uncertainty.",
    "Inventory write-downs negatively impacted our gross margin this quarter.",
    
    # Neutral statements
    "Revenue came in line with our expectations at $2.5 billion for the quarter.",
    "We continue to monitor the competitive landscape and adjust our strategy accordingly.",
    "Our investment in R&D remains consistent with historical levels at approximately 15% of revenue.",
    "We completed the previously announced acquisition and integration is proceeding as planned.",
    "Our geographic mix shifted slightly with APAC representing 35% of total revenue."
]

# Expected labels for validation
EXPECTED_LABELS = [
    'positive', 'positive', 'positive', 'positive', 'positive',
    'negative', 'negative', 'negative', 'negative', 'negative',
    'neutral', 'neutral', 'neutral', 'neutral', 'neutral'
]

print(f"Loaded {len(SAMPLE_STATEMENTS)} sample statements")

## 3. Sentiment Analysis Function

In [ ]:
def analyze_sentiment(texts, model, tokenizer, batch_size=8):
    """Analyze sentiment of financial texts using FinBERT"""
    results = []
    labels = ['positive', 'negative', 'neutral']
    
    model.eval()
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            # Tokenize
            inputs = tokenizer(
                batch_texts, 
                padding=True, 
                truncation=True, 
                max_length=512,
                return_tensors='pt'
            ).to(device)
            
            # Forward pass
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            
            # Get predictions
            for j, text in enumerate(batch_texts):
                prob_dict = {labels[k]: probs[j, k].item() for k in range(3)}
                predicted_label = labels[probs[j].argmax().item()]
                confidence = probs[j].max().item()
                
                # Composite sentiment score: positive - negative (range: -1 to 1)
                sentiment_score = prob_dict['positive'] - prob_dict['negative']
                
                results.append({
                    'text': text[:100] + '...' if len(text) > 100 else text,
                    'label': predicted_label,
                    'confidence': confidence,
                    'sentiment_score': sentiment_score,
                    'prob_positive': prob_dict['positive'],
                    'prob_negative': prob_dict['negative'],
                    'prob_neutral': prob_dict['neutral']
                })
    
    return pd.DataFrame(results)

# Analyze all statements
print("Analyzing sentiment...")
results_df = analyze_sentiment(SAMPLE_STATEMENTS, model, tokenizer)
results_df['expected'] = EXPECTED_LABELS
results_df['correct'] = results_df['label'] == results_df['expected']

print(f"\nAccuracy: {results_df['correct'].mean():.2%}")

## 4. Display Results

In [ ]:
# Display results table
display_cols = ['text', 'label', 'expected', 'confidence', 'sentiment_score', 'correct']
print("\nSentiment Analysis Results:")
print("="*100)

for idx, row in results_df.iterrows():
    status = "✓" if row['correct'] else "✗"
    score_bar = "█" * int((row['sentiment_score'] + 1) * 10)
    print(f"\n{idx+1}. {row['text'][:80]}...")
    print(f"   Predicted: {row['label']:<10} Expected: {row['expected']:<10} {status}")
    print(f"   Confidence: {row['confidence']:.2%}  Score: {row['sentiment_score']:+.3f}")
    print(f"   [Neg]{'─'*10}[Neutral]{'─'*10}[Pos]")
    print(f"         {score_bar}|")

## 5. Visualize Sentiment Distribution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sentiment score distribution
colors = {'positive': 'green', 'negative': 'red', 'neutral': 'gray'}
for label in ['positive', 'negative', 'neutral']:
    mask = results_df['expected'] == label
    axes[0, 0].scatter(
        results_df.loc[mask, 'sentiment_score'],
        results_df.loc[mask, 'confidence'],
        c=colors[label], label=label, s=100, alpha=0.7
    )
axes[0, 0].set_xlabel('Sentiment Score')
axes[0, 0].set_ylabel('Confidence')
axes[0, 0].set_title('Sentiment Score vs Confidence')
axes[0, 0].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Probability distribution
x = range(len(results_df))
axes[0, 1].bar(x, results_df['prob_positive'], label='Positive', color='green', alpha=0.7)
axes[0, 1].bar(x, results_df['prob_neutral'], bottom=results_df['prob_positive'], 
              label='Neutral', color='gray', alpha=0.7)
axes[0, 1].bar(x, results_df['prob_negative'], 
              bottom=results_df['prob_positive'] + results_df['prob_neutral'],
              label='Negative', color='red', alpha=0.7)
axes[0, 1].set_xlabel('Statement Index')
axes[0, 1].set_ylabel('Probability')
axes[0, 1].set_title('Sentiment Probability Distribution')
axes[0, 1].legend()

# Confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(results_df['expected'], results_df['label'], 
                      labels=['positive', 'neutral', 'negative'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 0],
           xticklabels=['Pos', 'Neu', 'Neg'],
           yticklabels=['Pos', 'Neu', 'Neg'])
axes[1, 0].set_xlabel('Predicted')
axes[1, 0].set_ylabel('Actual')
axes[1, 0].set_title('Confusion Matrix')

# Sentiment score histogram
axes[1, 1].hist(results_df['sentiment_score'], bins=15, edgecolor='black', alpha=0.7)
axes[1, 1].axvline(x=0, color='red', linestyle='--', label='Neutral')
axes[1, 1].set_xlabel('Sentiment Score')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Sentiment Score Distribution')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Simulate Trading Signal Generation

In [ ]:
def generate_trading_signal(sentiment_score, confidence, threshold=0.3):
    """
    Generate trading signal from sentiment analysis.
    
    Returns:
        signal: 'BUY', 'SELL', or 'HOLD'
        strength: Signal strength (0-1)
    """
    # Confidence-weighted sentiment
    weighted_score = sentiment_score * confidence
    
    if weighted_score > threshold:
        return 'BUY', min(abs(weighted_score), 1.0)
    elif weighted_score < -threshold:
        return 'SELL', min(abs(weighted_score), 1.0)
    else:
        return 'HOLD', 0.0

# Generate signals for all statements
signals = []
for _, row in results_df.iterrows():
    signal, strength = generate_trading_signal(row['sentiment_score'], row['confidence'])
    signals.append({'signal': signal, 'strength': strength})

signals_df = pd.DataFrame(signals)
results_df = pd.concat([results_df, signals_df], axis=1)

# Display trading signals
print("\nTrading Signals:")
print("="*80)
for idx, row in results_df.iterrows():
    signal_icon = {'BUY': '🟢', 'SELL': '🔴', 'HOLD': '⚪'}[row['signal']]
    strength_bar = '█' * int(row['strength'] * 10)
    print(f"{idx+1}. {signal_icon} {row['signal']:<5} | Strength: {strength_bar:<10} ({row['strength']:.2f})")
    print(f"   {row['text'][:60]}...")
    print()

## 7. Simulate Earnings Call Analysis Pipeline

In [ ]:
def analyze_earnings_call(paragraphs, model, tokenizer):
    """
    Analyze a full earnings call transcript.
    
    Args:
        paragraphs: List of text paragraphs from earnings call
        
    Returns:
        overall_sentiment: Aggregated sentiment score
        section_results: DataFrame with per-section analysis
    """
    # Analyze each paragraph
    section_results = analyze_sentiment(paragraphs, model, tokenizer)
    
    # Aggregate sentiment (weighted by confidence)
    weights = section_results['confidence']
    overall_sentiment = np.average(section_results['sentiment_score'], weights=weights)
    
    # Key metrics
    positive_ratio = (section_results['label'] == 'positive').mean()
    negative_ratio = (section_results['label'] == 'negative').mean()
    avg_confidence = section_results['confidence'].mean()
    
    return {
        'overall_sentiment': overall_sentiment,
        'positive_ratio': positive_ratio,
        'negative_ratio': negative_ratio,
        'avg_confidence': avg_confidence,
        'section_results': section_results
    }

# Simulate analyzing multiple companies
COMPANY_CALLS = {
    'TechCorp': [
        "Revenue growth exceeded our expectations driven by cloud adoption.",
        "We're seeing strong momentum in our AI products.",
        "Customer retention improved significantly this quarter.",
        "We're raising guidance for the full year."
    ],
    'RetailInc': [
        "Same-store sales declined due to reduced foot traffic.",
        "Inventory levels remain elevated, requiring markdowns.",
        "We're implementing cost reduction measures.",
        "Consumer sentiment remains challenging."
    ],
    'BankCo': [
        "Net interest income was in line with expectations.",
        "Credit quality remains stable across our portfolio.",
        "We continue to invest in digital banking capabilities.",
        "Capital ratios exceed regulatory requirements."
    ]
}

print("Analyzing Earnings Calls...")
print("="*60)

company_signals = []
for company, paragraphs in COMPANY_CALLS.items():
    analysis = analyze_earnings_call(paragraphs, model, tokenizer)
    signal, strength = generate_trading_signal(
        analysis['overall_sentiment'], 
        analysis['avg_confidence']
    )
    
    company_signals.append({
        'company': company,
        'sentiment': analysis['overall_sentiment'],
        'confidence': analysis['avg_confidence'],
        'positive_ratio': analysis['positive_ratio'],
        'signal': signal,
        'strength': strength
    })
    
    signal_icon = {'BUY': '🟢', 'SELL': '🔴', 'HOLD': '⚪'}[signal]
    print(f"\n{company}:")
    print(f"  Overall Sentiment: {analysis['overall_sentiment']:+.3f}")
    print(f"  Positive Ratio: {analysis['positive_ratio']:.0%}")
    print(f"  Confidence: {analysis['avg_confidence']:.2%}")
    print(f"  Signal: {signal_icon} {signal} (strength: {strength:.2f})")

# Ranking
signals_ranked = pd.DataFrame(company_signals).sort_values('sentiment', ascending=False)
print("\n" + "="*60)
print("\nRanking by Sentiment:")
for i, (_, row) in enumerate(signals_ranked.iterrows(), 1):
    print(f"  {i}. {row['company']:<12} Sentiment: {row['sentiment']:+.3f} | Signal: {row['signal']}")

## 8. Attention Visualization

In [ ]:
def get_attention_weights(text, model, tokenizer):
    """Extract attention weights for visualization"""
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)
    
    # Get attention from last layer, average over heads
    attention = outputs.attentions[-1][0].mean(dim=0)  # (seq_len, seq_len)
    
    # Get tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # Get CLS token attention (attention to other tokens)
    cls_attention = attention[0, 1:-1].cpu().numpy()  # Skip [CLS] and [SEP]
    tokens = tokens[1:-1]  # Skip special tokens
    
    return tokens, cls_attention

# Visualize attention for a sample text
sample_text = "We are extremely pleased to report record revenue growth and raising our full-year guidance."

tokens, attention = get_attention_weights(sample_text, model, tokenizer)

# Normalize attention for visualization
attention_norm = (attention - attention.min()) / (attention.max() - attention.min())

plt.figure(figsize=(14, 4))
plt.bar(range(len(tokens)), attention_norm, color='steelblue')
plt.xticks(range(len(tokens)), tokens, rotation=45, ha='right')
plt.ylabel('Normalized Attention')
plt.title(f'Attention Weights: "{sample_text[:50]}..."')
plt.tight_layout()
plt.show()

# Highlight top attention tokens
top_k = 5
top_indices = np.argsort(attention)[-top_k:]
print(f"\nTop {top_k} attended tokens:")
for idx in reversed(top_indices):
    print(f"  '{tokens[idx]}': {attention[idx]:.4f}")

## Summary

This notebook demonstrated:

1. **FinBERT**: Pre-trained transformer model for financial sentiment analysis
2. **Sentiment Scoring**: Convert text to quantitative sentiment signals
3. **Trading Signal Generation**: Threshold-based signal generation from sentiment
4. **Earnings Call Analysis**: Aggregate paragraph-level sentiment for company scoring
5. **Attention Visualization**: Understanding which words drive sentiment predictions

### Key Insights:
- FinBERT effectively classifies financial sentiment with high accuracy
- Confidence-weighted aggregation improves signal quality
- Key financial terms (revenue, growth, guidance) receive high attention weights

### Extensions to Try:
- Fine-tune FinBERT on your own labeled earnings call data
- Combine with price data to measure predictive power
- Add named entity recognition for ticker extraction
- Implement real-time earnings call streaming analysis